# Introduction

This program performs analysis and visualization of openLCA results from an LCIA calculation, starting with an excel file export. This program assumes that you've already compiled an excel file of the unit processes exported from your openLCA database, with ISIC categories assigned to each process. This excel file should be named "database_process_summary_added.xlsx" with the corresponding file path.

# Setup

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Set up variables

lca_results_file_path = r'C:\Users\laura\OneDrive\Documents\UT_graduate\WEG_graduate_research\Hydrogen_fugitive_emissions_natural_h2\openLCA_ecoinvent\Hydrogen_gas__at_processing__production_mixture__to_consumer__kg___US__85__mol_H2____H2_Average___CH4_Capture.xlsx'
sheet_name = 'Direct impact contributions'
